In [47]:
import json
import pandas as pd
import re
from collections import Counter



In [48]:
# 📌 3. JSON 데이터 로드
file_path = "../data/news_with_keywords1.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 📌 4. 데이터프레임 변환
df = pd.DataFrame(data)

In [49]:
df

,title,link,date,content,source,quarter,cleaned_content,tokenized_content,keywords
0,朝鲜试射导弹亚太骤起波澜,https://www.gmw.cn/01gmrb/2006-07/12/content_4...,2006-07-12,7月5日，朝鲜连续发射7枚导弹，引起国际社会震惊。近日，美日在安理会积极推动制裁朝鲜的议案，...,光明网,2006Q3,7月5日朝鲜连续发射7枚导弹引起国际社会震惊。近日美日在安理会积极推动制裁朝鲜的议案并加紧协...,朝鲜 连续 发射 7 枚 导弹 引起 国际 社会 震惊 。 美 日 安理会 推动 制裁 朝鲜...,"朝鲜 (0.3848), 导弹 (0.3582), 发射 (0.2838), 制裁 (0.2..."
1,美朝关系开始解冻,https://www.gmw.cn/01gmrb/2007-03/08/content_5...,2007-03-08,被外界视为“破冰之旅”的朝鲜副外相金桂冠3月5至6日在纽约与美国助理国务卿希尔就两国关系正常...,光明网,2007Q1,被外界视为破冰之旅的朝鲜副外相金桂冠3月5至6日在纽约与美国助理国务卿希尔就两国关系正常化进...,外界 视为 破冰 之 旅 朝鲜 副外相 金桂冠 5 纽约 美国 助理 国务卿 希尔 关系 正...,"会谈 (0.4823), 正常化 (0.3859), 朝鲜 (0.2748), 关系 (0...."
2,美日韩间谍云集朝鲜周边空中侦察地面窃听 - 国际经济,http://intl.ce.cn/zj/200809/19/t20080919_16855...,2008-09-19,神秘，这是朝鲜留给外界的印象。 正因为神秘，一些国家的情报机构费尽心机，通过投放卫星、派遣侦...,中国经济网,2008Q3,神秘这是朝鲜留给外界的印象。 正因为神秘一些国家的情报机构费尽心机通过投放卫星派遣侦察机甚至...,神秘 朝鲜 留给 外界 印象 。 正 神秘 国家 情报 费尽心机 投放 卫星 派遣 侦察机 ...,"情报 (0.5919), 朝鲜 (0.5456), 侦察 (0.2265), 卫星 (0.1..."
3,美日韩紧盯朝鲜射导弹(图) - 国际经济,http://intl.ce.cn/zj/200902/11/t20090211_18171...,2009-02-11,改进型“大浦洞2号”据说可攻击美本土，甚至可能具有搭载核弹头技术 朝鲜政府最近频繁发出朝韩关...,中国经济网,2009Q1,改进型大浦洞2号据说可攻击美本土甚至可能具有搭载核弹头技术 朝鲜政府最近频繁发出朝韩关系恶化...,改进型 大浦洞 2 据说 攻击 美 本土 具有 搭载 核 弹头 技术 朝鲜 最近 频繁 发出...,"导弹 (0.6370), 朝鲜 (0.4398), 试射 (0.2929), 公里 (0.2..."
4,美国朝鲜问题特使称美愿与朝对话,https://world.huanqiu.com/article/9CaKrnJlEWs,2009-03-07,新华网首尔3月7日电 (记者李拯宇 干玉兰) 美国朝鲜问题特使斯蒂芬·博斯沃思7日在韩国说，...,环球网,2009Q1,新华网首尔3月7日电 记者李拯宇 干玉兰 美国朝鲜问题特使斯蒂芬博斯沃思7日在韩国说美国愿意...,新华网 首尔 电 李拯宇 干玉兰 美国 朝鲜 问题 特使 斯蒂芬博斯沃思 韩国 美国 朝鲜 ...,"朝鲜 (0.5444), 对话 (0.3843), 美国 (0.3404), 抵达 (0.3..."
...,...,...,...,...,...,...,...,...,...
1034,詹德斌：韩国政局变动，美国要做的不应是施压,https://hqtime.huanqiu.com/share/article/4Ky1A...,2025-01-07,美国国务卿布林肯1月5日抵达韩国。这看上去似乎是一次礼仪性道别之旅，但防止韩国“后弹劾政局”...,环球网,2025Q1,美国国务卿布林肯1月5日抵达韩国。这看上去似乎是一次礼仪性道别之旅但防止韩国后弹劾政局对美韩...,美国 国务卿 布林肯 抵达 韩国 。 看上去 似乎 礼仪性 道别 之 旅 防止 韩国 后 弹...,"外交 (0.4220), 韩国 (0.2513), 尹锡悦 (0.2288), 价值观 (0..."
1035,朝鲜谴责美韩军事挑衅导致地区局势恶化,http://www.xinhuanet.com/20250126/55a39b230ccf...,2025-01-26,新华社首尔1月26日电 据朝中社26日报道，朝鲜外务省对外政策室长当天发表谈话，谴责美韩近期...,新华网,2025Q1,新华社首尔1月26日电 据朝中社26日报道朝鲜外务省对外政策室长当天发表谈话谴责美韩近期对朝...,首尔 电 朝中社 朝鲜 外务省 对外 政策 室长 谈话 谴责 美 韩 朝 军事 挑衅 行为 ...,"谈话 (0.5243), 行为 (0.2876), 朝方 (0.2717), 挑衅 (0.2..."
1036,朝鲜外务省：美韩军事挑衅行为导致地区局势恶化 - 国际频道,https://world.gmw.cn/2025-01/26/content_378203...,2025-01-26,中新网1月26日电据朝中社报道，当地时间1月26日，朝鲜外务省对外政策室长发表谈话，谴责美韩...,光明网,2025Q1,中新网1月26日电据朝中社报道当地时间1月26日朝鲜外务省对外政策室长发表谈话谴责美韩近期对...,中新网 电 朝中社 朝鲜 外务省 对外 政策 室长 谈话 谴责 美 韩 朝 军事 挑衅 行为...,"谈话 (0.5955), 行为 (0.2723), 朝方 (0.2572), 挑衅 (0.2..."
1037,石破茂与特朗普会面，美日“小集团”谋划“大算盘” | 国际识局,http://intl.ce.cn/qqss/202502/09/t20250209_392...,2025-02-09,中新网2月9日电(记者 孟湘君)特朗普当选新一任美国总统后，意大利、以色列等国领导人先后与其...,中国经济网,2025Q1,中新网2月9日电记者 孟湘君特朗普当选新一任美国总统后意大利以色列等国领导人先后与其会面。近...,中新网 电 孟湘君 特朗普 当选 新 任 美国 总统 后 意大利 以色列 先后 会面 。 赶...,"特朗普 (0.5579), 科技 (0.2335), 日本 (0.1935), 同盟 (0...."


In [50]:

# 분기별로 상위 10개 키워드 추출
result = []

for quarter, group in df.groupby("quarter"):
    all_keywords = []

    # 정규 표현식을 사용하여 "키워드 (점수)" 형식에서 키워드와 점수 분리
    for keywords_str in group["keywords"]:
        matches = re.findall(r"([\w\u4e00-\u9fff]+) \(([\d.]+)\)", keywords_str)
        all_keywords.extend([(kw, float(score)) for kw, score in matches])

    # 점수 기준 내림차순 정렬 후 상위 10개 선택
    top_keywords = sorted(all_keywords, key=lambda x: x[1], reverse=True)[:10]

    # 결과 저장
    for keyword, score in top_keywords:
        result.append({"Quarter": quarter, "Keyword": keyword, "Score": score})

# DataFrame 변환 및 출력
df_result = pd.DataFrame(result)
print(df_result)


    Quarter Keyword   Score
0    2006Q3      朝鲜  0.3848
1    2006Q3      导弹  0.3582
2    2006Q3      发射  0.2838
3    2006Q3      制裁  0.2532
4    2006Q3      日本  0.1957
..      ...     ...     ...
635  2025Q1      中国  0.3082
636  2025Q1      台湾  0.2937
637  2025Q1      行为  0.2876
638  2025Q1     特朗普  0.2856
639  2025Q1     布林肯  0.2733

[640 rows x 3 columns]


In [51]:
df_result

,Quarter,Keyword,Score
0,2006Q3,朝鲜,0.3848
1,2006Q3,导弹,0.3582
2,2006Q3,发射,0.2838
3,2006Q3,制裁,0.2532
4,2006Q3,日本,0.1957
...,...,...,...
635,2025Q1,中国,0.3082
636,2025Q1,台湾,0.2937
637,2025Q1,行为,0.2876
638,2025Q1,特朗普,0.2856


In [52]:
df["year"] = df["quarter"].str[:4].astype(int)  # "2006Q3" → 2006

# 📌 2006~2009년 데이터를 "2006-2009Q"로 그룹화, 나머지는 기존 분기(YYYYQ#) 유지
df["input_quarter"] = df["quarter"].apply(lambda x: "2006-2009Q" if int(x[:4]) in range(2006, 2010) else x)
df["input_year"] = df["year"].apply(lambda x: "2006-2009" if 2006 <= x <= 2009 else str(x))


In [54]:
import pandas as pd
import re
from collections import defaultdict, Counter

# 결과 저장
result_text = ""

for input_quarter, group in df.groupby("input_quarter"):
    keyword_scores = defaultdict(list)
    num_documents = len(group)  # 해당 분기의 문서 수 계산

    # 정규 표현식을 사용하여 키워드와 점수를 추출
    for keywords_str in group["keywords"]:
        matches = re.findall(r"([\w\u4e00-\u9fff]+) \(([\d.]+)\)", keywords_str)
        for kw, score in matches:
            keyword_scores[kw].append(float(score))  # 키워드별 점수 리스트 저장

    # 키워드 빈도수 계산
    keyword_counts = Counter({kw: len(scores) for kw, scores in keyword_scores.items()})

    # 키워드별 평균 점수 계산
    keyword_avg_scores = {kw: sum(scores) / len(scores) for kw, scores in keyword_scores.items()}

    # 정렬 기준: 1) 빈도수 내림차순, 2) 평균 점수 내림차순
    sorted_keywords = sorted(keyword_counts.keys(), key=lambda kw: (-keyword_counts[kw], -keyword_avg_scores[kw]))

    # 상위 10개 키워드 선택
    top_keywords = [(kw, keyword_counts[kw], round(keyword_avg_scores[kw], 4)) for kw in sorted_keywords[:10]]

    # 원하는 출력 형식에 맞춰 문자열 생성
    result_text += f"{input_quarter}분기 중요 키워드 (문서 수: {num_documents}개):\n"
    result_text += ", ".join([f"{kw}({count}회,{avg_score}점)" for kw, count, avg_score in top_keywords]) + "\n\n"

# 결과 출력
print(result_text)


2006-2009Q분기 중요 키워드 (문서 수: 14개):
朝鲜(12회,0.3482점), 发射(8회,0.286점), 卫星(4회,0.2783점), 导弹(3회,0.4579점), 安理会(3회,0.329점), 美国(3회,0.245점), 火箭(2회,0.3573점), 会谈(2회,0.3292점), 声明(2회,0.3268점), 对话(2회,0.2808점)

2010Q1분기 중요 키워드 (문서 수: 1개):
演习(1회,0.4686점), 参加(1회,0.361점), 联合(1회,0.3132점), 军演(1회,0.3054점), 海外(1회,0.2414점), 军事(1회,0.2276점), 机会(1회,0.2151점), 美军(1회,0.202점), 军队(1회,0.1991점), 韩国(1회,0.1719점)

2010Q3분기 중요 키워드 (문서 수: 2개):
军演(2회,0.4596점), 演习(2회,0.2391점), 中国(1회,0.4765점), 联合(1회,0.3605점), 围堵(1회,0.2715점), 美国(1회,0.2637점), 南海(1회,0.2067점), 战斗机(1회,0.1865점), 东南亚(1회,0.1837점), 势力(1회,0.1702점)

2010Q4분기 중요 키워드 (문서 수: 8개):
朝鲜(6회,0.2436점), 会谈(5회,0.2652점), 日本(4회,0.2555점), 美国(4회,0.1994점), 外长(3회,0.3591점), 中国(3회,0.2447점), 半岛(3회,0.1876점), 华盛顿(3회,0.1694점), 问题(2회,0.1752점), 主席(2회,0.1651점)

2011Q1분기 중요 키워드 (문서 수: 6개):
朝鲜(3회,0.2786점), 日本(3회,0.2001점), 军演(2회,0.5202점), 协定(2회,0.3402점), 军事(2회,0.3105점), 签署(2회,0.2958점), 美国(2회,0.2299점), 合作(2회,0.2076점), 韩国(2회,0.1827점), 侵略(1회,0.4535점)

2011Q3분기 중요 키워드 (문서 수: 1개):
首脑(1회,0.3314점), 问题(1회,0.287